In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

import cbmos
import cbmos.force_functions as ff
import cbmos.solvers.euler_forward as ef
import cbmos.solvers.euler_backward as eb
import cbmos.cell as cl

In [2]:
plt.style.use('seaborn-whitegrid')
plt.style.use('tableau-colorblind10')
params = {'legend.fontsize': 'xx-large',
          'figure.figsize': (6.75, 5),
          'lines.linewidth': 3.0,
         'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large',
          'legend.fontsize': 'xx-large',
         'font.size': 11,
          'font.family': 'serif',
          "mathtext.fontset": "dejavuserif",
         'axes.titlepad': 12,
        'axes.labelpad': 12}
plt.rcParams.update(params)

In [3]:
# Simulation parameters
s = 1.0    # rest length
tf = 1.0  # final time
rA = 1.5   # maximum interaction distance

seed=17

params_cubic = {"mu": 5.70, "s": s, "rA": rA}
muR = 9.1
ratio = 0.21
params_poly = {'muA': muR*ratio, 'muR': muR, 'rA': rA, 'rR': 1.0/(1.0-np.sqrt(ratio)/3.0), 'n': 1.0, 'p': 1.0}
mu_gls = 1.95
params_gls = {'mu': mu_gls, 'a': 7.51}
params = {'cubic': params_cubic, 'pw. quad.': params_poly, 'GLS': params_gls}

In [4]:
force_names = ['cubic', 'pw. quad.', 'GLS']
solver_names = ['EF', 'EB']

defcolors = plt.rcParams['axes.prop_cycle'].by_key()['color']
colors = {'cubic': defcolors[0], 'pw. quad.': defcolors[5], 'GLS': defcolors[6]}
linestyles = {'cubic': '-', 'pw. quad.': '--', 'GLS': '-.'}

In [5]:
models_ef = {'pw. quad.': cbmos.CBMModel(ff.PiecewisePolynomial(), ef.solve_ivp, 1), 
             'cubic': cbmos.CBMModel(ff.Cubic(), ef.solve_ivp, 1),
             'GLS': cbmos.CBMModel(ff.Gls(), ef.solve_ivp, 1)}
models_eb = {'pw. quad.': cbmos.CBMModel(ff.PiecewisePolynomial(), eb.solve_ivp, 1), 
             'cubic': cbmos.CBMModel(ff.Cubic(), eb.solve_ivp, 1),
             'GLS': cbmos.CBMModel(ff.Gls(), eb.solve_ivp, 1)}
model_dicts = {'EF': models_ef, 'EB': models_eb}

In [6]:
# Initial condition
cell_list = [cl.Cell(0, [0], proliferating=False), cl.Cell(1, [0.3], proliferating=False)]

In [7]:
dt = 1e-4
t_data_ref = np.arange(0, tf, dt)
#t_data_ref = np.linspace(0, tf, 10000+1)

#t_data_ref=[0,tf]
ref_sol_dicts = {}
for solver in solver_names:
    models = model_dicts[solver]
    ref_sols = {}
    for force in force_names: 
        #params_solver = {'EF':{'dt': 0.0001}, 'EB': {'dt': 0.0001, 'jacobian': models[force].jacobian}}
        params_solver = {'EF':{'dt': dt}, 'EB': {'dt': dt}}
        ref_sols[force] = models[force].simulate(cell_list, t_data_ref, params[force], params_solver[solver], seed=seed)
    ref_sol_dicts[solver] = ref_sols

In [8]:
# Check eigenvalues at beginning
y0 = np.array([cell.position for cell in cell_list]).reshape(-1)
for force in force_names:
    A = model_dicts['EF'][force].jacobian(y0, params[force])
    w,v = np.linalg.eig(A)
    print(w)
    print(np.all(w < 0))

[-3.55271368e-15 -3.55680000e+01]
True
[  0.         -18.92432326]
False
[  0. -13.]
False


In [19]:
# Check eigenvalues in the middle
for force in force_names:
    t_data, history = ref_sol_dicts['EF'][force]
    y = np.array([cell.position for cell in history[200]]).reshape(-1)
    A = model_dicts['EF'][force].jacobian(y, params[force])
    w,v = np.linalg.eig(A)
    print(w)
    print(np.all(w < 0))

[  0.         -24.48600043]
False
[ 8.88178420e-16 -1.59878089e+01]
False
[-8.88178420e-16 -1.01664802e+01]
True


In [13]:
t_data, cell_coords = ref_sol_dicts['EF'][force]

In [ ]:
cell_coords

In [ ]:
ref_sol_dicts['EF'][force]